In [ ]:
import pandas as pb
data = pb.read_excel('updFAKSexp.xlsx')
print(data.shape)

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pb

input_tokens = [(input_text) for input_text in data.iloc[:, 1:].iterrows()]
print(len(input_tokens))
output_tokens = [(output_text) for output_text in data.iloc[:, 0]]

par_tokens = [(output_tokens[i], input_tokens[i])for i in range(len(input_tokens))]

train_texts, val_and_test_texts = train_test_split(par_tokens, test_size=0.2, random_state=42) #måste läsa lite om random_state
val_texts, test_texts = train_test_split(val_and_test_texts, test_size=0.5, random_state=42)

output_tokens_from_par_train = [str(item) for item in train_texts]
output_tokens_from_par_val = [str(item) for item in val_texts]
output_tokens_from_par_test = [str(item) for item in test_texts]
input_tokens_from_par_train = [str(item) for item in train_texts]
input_tokens_from_par_val = [str(item) for item in val_texts]
input_tokens_from_par_test = [str(item) for item in test_texts]

PatientJournalText_train_small = [str(item) for item in output_tokens_from_par_train[:80]]
PatientJournalText_val_small = [str(item) for item in output_tokens_from_par_val[:10]]
PatientJournalText_test_small = [str(item) for item in output_tokens_from_par_test[:10]]
InputData_train_small = [str(item) for item in input_tokens_from_par_train[:80]]
InputData_val_small = [str(item) for item in input_tokens_from_par_val[:10]]
InputData_test_small = [str(item) for item in input_tokens_from_par_test[:10]]

In [ ]:
import pandas as pb
datasmall = pb.read_excel('LitenData.xlsx')
print(datasmall.shape)

from sklearn.model_selection import train_test_split
import pandas as pb

input_tokens = [(input_text) for input_text in datasmall.iloc[:, 1:].iterrows()]
print(len(input_tokens))
output_tokens = [(output_text) for output_text in datasmall.iloc[:, 0]]

par_tokens = [(output_tokens[i], input_tokens[i])for i in range(len(input_tokens))]

train_texts, val_and_test_texts = train_test_split(par_tokens, test_size=0.2, random_state=42) #måste läsa lite om random_state
val_texts, test_texts = train_test_split(val_and_test_texts, test_size=0.5, random_state=42)

output_tokens_from_par_train_small = [str(item) for item in train_texts]
output_tokens_from_par_val_small = [str(item) for item in val_texts]
output_tokens_from_par_test_small = [str(item) for item in test_texts]
input_tokens_from_par_train_small = [str(item) for item in train_texts]
input_tokens_from_par_val_small = [str(item) for item in val_texts]
input_tokens_from_par_test_small = [str(item) for item in test_texts]


In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, pipeline, AutoTokenizer, AutoModelForCausalLM

model_name = "AI-Sweden-Models/gpt-sw3-126m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, PatientJournalText_small, InputData_small, tokenizer):
        self.PatientJournalText_small = PatientJournalText_small
        self.InputData_small = InputData_small
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.InputData_small)

    def __getitem__(self, idx):        
        PatientJournalText_small = self.PatientJournalText_small[idx]
        InputData_small = self.InputData_small[idx]

        # Encode input sequence and labels using tokenizer
        inputs = self.tokenizer.encode_plus(
            InputData_small,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length = 320
            
        )

        labels = self.tokenizer.encode_plus(
            PatientJournalText_small,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length = 320
            
        )

        return {
            "input_ids": inputs.input_ids.flatten(),
            "labels": labels.input_ids.flatten()
        }




AllData_train_small = MyDataset(output_tokens_from_par_train_small, input_tokens_from_par_train_small, tokenizer)
AllData_val_small = MyDataset(output_tokens_from_par_val_small, input_tokens_from_par_val_small, tokenizer)
AllData_test_small = MyDataset(output_tokens_from_par_test_small, input_tokens_from_par_test_small, tokenizer)

In [ ]:
import torch
from transformers import LineByLineTextDataset, pipeline, AutoTokenizer, AutoModelForCausalLM

model_name = "AI-Sweden-Models/gpt-sw3-126m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


class MyDataset():

    def __init__(self, text_pair, tokenizer):
        self.text_pair = text_pair
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.text_pair)

    def Tokinizering(self):
        return LineByLineTextDataset(tokenizer=tokenizer, text_pairs = self.text_pair)


AllData_train_small = MyDataset(train_texts, tokenizer)
AllData_val_small = MyDataset(val_texts, tokenizer)
AllData_test_small = MyDataset(test_texts, tokenizer)

In [ ]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForCausalLM
import numpy as np
import evaluate



training_args = TrainingArguments(
    #output_dir="test_trainer", 
    evaluation_strategy="epoch",
    eval_steps=500,
    num_train_epochs=2,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=6,
    #gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    logging_steps=50,
    save_steps=50
    #logging_dir="./logs")
    #metric = evaluate.load("accuracy")
)

'''def compute_metrics(eval_pred):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)'''

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset= AllData_train_small,
    eval_dataset= AllData_val_small,
    #compute_metrics= compute_metrics,
)

trainer.train()
trainer.save_model("/mnt/d1/KEX/.myenv/GPT-sw3-126m.my_own")